# 02 — Code Examples (Runnable)

This notebook provides runnable tool stubs and a minimal agent loop. It includes a small Memory class copied locally so the notebook is self-contained and can be run without importing from other notebooks.

In [1]:
# Small copy of Memory class (self-contained)
class Memory:
    def __init__(self, char_limit=500):
        self.char_limit = int(char_limit)
        self.history = []
        self._total_chars = 0
    def add(self, role, content):
        item = {"role": role, "content": content}
        self.history.append(item)
        self._total_chars += len(content)
        while self._total_chars > self.char_limit and self.history:
            removed = self.history.pop(0)
            self._total_chars -= len(removed['content'])
    def summarize(self, max_items=5):
        return ' | '.join(m['content'] for m in self.history[-max_items:])
    def to_system_prompt(self):
        return 'Memory summary: ' + self.summarize() if self.history else ''

mem = Memory(char_limit=300)
mem.add('user', 'I like concise summaries')
mem.add('assistant', 'Will keep it brief')
print('Memory summary:', mem.summarize())

Memory summary: I like concise summaries | Will keep it brief


## Tool Stubs
Tools are small callable functions (or classes) that an agent can use to get information or perform actions. We'll create stubs to show the pattern.

In [2]:
# Tool stub examples with structured returns
def search_wiki(query):
    """Pretend to search a knowledge base and return a short summary and a URL."""
    return {"source": "wiki", "query": query, "summary": f"Summary for: {query}", "url": f"https://example.org/wiki/{query.replace(' ', '_')}"}

def run_sql(query):
    """Pretend to run SQL and return a small table as list of dicts."""
    # Very small fake table for demo
    return {"rows": [{"id": 1, "value": 42}], "query": query}

# Example usage
print(search_wiki('LangGraph'))
print(run_sql('SELECT 1'))

{'source': 'wiki', 'query': 'LangGraph', 'summary': 'Summary for: LangGraph', 'url': 'https://example.org/wiki/LangGraph'}
{'rows': [{'id': 1, 'value': 42}], 'query': 'SELECT 1'}


## Agent Loop (Simplified)
We'll show a minimal loop: read user input, choose a tool, call it, and return the result. In production you would let the LLM decide which tool to call using function calling or a decision step. This example returns both a text response and the raw tool output when available.

In [3]:
import re

def minimal_agent(user_input, memory=None):
    user_lower = user_input.lower()
    # dispatch rules
    if 'wiki' in user_lower or re.search(r'search\s+wiki', user_lower):
        out = search_wiki(user_input)
        text = f"I searched the wiki and found: {out['summary']} (source: {out['url']})"
        if memory is not None:
            memory.add('assistant', text)
        return text, out
    if 'sql' in user_lower or re.search(r'select|from|where', user_lower):
        out = run_sql(user_input)
        text = f"SQL ran and returned {len(out['rows'])} rows."
        if memory is not None:
            memory.add('assistant', text)
        return text, out
    # default reply
    fallback = 'I can search wiki or run sql. Try adding the word wiki or a SQL statement.'
    if memory is not None:
        memory.add('assistant', fallback)
    return fallback, None

# Demo runs
mem = Memory(char_limit=300)
print(minimal_agent('Search wiki for LangGraph', memory=mem))
print(minimal_agent('Run SQL: SELECT id, value FROM metrics', memory=mem))
print('Memory after interactions:', mem.summarize())

('I searched the wiki and found: Summary for: Search wiki for LangGraph (source: https://example.org/wiki/Search_wiki_for_LangGraph)', {'source': 'wiki', 'query': 'Search wiki for LangGraph', 'summary': 'Summary for: Search wiki for LangGraph', 'url': 'https://example.org/wiki/Search_wiki_for_LangGraph'})
('SQL ran and returned 1 rows.', {'rows': [{'id': 1, 'value': 42}], 'query': 'Run SQL: SELECT id, value FROM metrics'})
Memory after interactions: I searched the wiki and found: Summary for: Search wiki for LangGraph (source: https://example.org/wiki/Search_wiki_for_LangGraph) | SQL ran and returned 1 rows.


## Exercises
- Extend the agent to include a confidence score when selecting tools and show the score in the textual reply.
- Replace the keyword-based dispatch with a simple rule-engine that matches regex patterns to tools (hint: a list of (pattern, tool) pairs).

End of `02_code_examples.ipynb`. Next steps: create more focused notebooks for `RAG`, `Tooling`, and `LangGraph` workflows.